##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
## tk文件导入模块

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def save_model_all_laws(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该所有法条的数量表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass
    
def save_model_single_laws(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该含有某种法律的数量表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该表格的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
            print("存储完毕")
        else:
            print("未能存储")
            pass
    except Exception as e:
        pass

##### 第三模块(核心提取器模块：必须运行）

In [3]:
### 文本清理器1
def clean(corpus_none_n):
    """
    param corpus_none_n:不含none的文本列表
    return table_caipanwenshu
    """
    ## 正则表达式部分开始 --------------------------------------------------------------------
    results = corpus_none_n
    ### 原告中的三类可能
    pattern_plaintiff = re.compile(r"原告：([\u4e00-\u9fa5].*?)，") # 原告第一类可能
    pattern_plaintiff_2 = re.compile(r"原告：([\u4e00-\u9fa5].*?)。") # 原告第二类可能
    pattern_plaintiff_3 = re.compile(r"原告.*?，") # 原告第三类可能

    ### 被告中的三类可能
    pattern_defender = re.compile(r"被告：.*?，")
    pattern_defender_2 = re.compile(r"被告：.*?。")

    ### 原告出生年龄
    pattern_plaintiff_birthday = re.compile(r"原告：.*\d.*?出生，")

    ### 被告出生年龄
    pattern_defender_birthday = re.compile(r"被告：.*\d.*?出生，")

    ### 立案时间
    pattern_starttime = re.compile(r"，本院于.*\d.立案后，")

    ### 诉讼请求
    pattern_claims = re.compile(r"诉讼请求：.*?。")
    pattern_claims_2 = re.compile(r"诉称，.*本院认为")

    ### 事实和理由
    pattern_facts = re.compile(r"事实和理由：.*。")

    ### 辩护事实
    pattern_defence = re.compile(r"辩称：.*")
    pattern_defence_2 = re.compile(r"辩称，.*")

    ### 法院认定事实
    pattern_court_fact_1 = re.compile(r"认定事实如下：.*。本院")
    pattern_court_fact_2 = re.compile(r"经审理查明：.*。本院")
    pattern_court_fact_3 = re.compile(r"经审理查明，.*。本院")

    ### 法院对事实的判定
    pattern_court_advise = re.compile(r"本院认为.*")

    ### 法院的意见
    pattern_court_attitude = re.compile(r"综上所述.*?。")

    ### 法条
    pattern_laws_1 = re.compile(r"《.*?》.*?条")
    pattern_laws_2 = re.compile(r"《.*?》.*?条，")
    pattern_laws_3 = re.compile(r"(《.*?[^》]*条)")
    pattern_laws_4 = re.compile(r"《.*?》")

    ### 判决
    pattern_judgement = re.compile("判决如下:.*。") #需要使用去掉\n的文档
    
    ## 正则表达式部分结束 --------------------------------------------------------------------
    ## 搜索原告
    def find_plaintiffs(text,pattern1,pattern2,pattern3):
        try:
            if re.search(pattern1,text):
                piece = re.search(pattern1,text).group().strip(":原告：，")
                return piece
            elif re.search(pattern2,text):
                piece = re.search(pattern2,text).group().strip(":原告：，")
                return piece
            elif re.search(pattern3,text):
                piece = re.search(pattern3,text).group().strip(":原告：，")
                return piece
            else:
                return None
        except Exception as e:
            pass

    ## 搜索被告
    def find_defender(text,pattern1,pattern2):
        try:
            if re.search(pattern1,text):
                piece = re.search(pattern1,text).group().strip("被告：")
                return piece
            elif re.search(pattern2,text):
                piece = re.search(pattern2,text).group().strip("被告：")
                return piece
            else:
                return None
        except Exception as e:
            pass

    ## 搜索原告出生年龄
    def find_plaintiffs_birthday(text,pattern):
        try:
            if re.search(pattern,text):
                piece = re.search(pattern,text).group().strip("，")
                text = re.sub("原告：.*，","",piece).strip("出生")
                return text
            else:
                return None
        except Exception as e:
            pass

    ## 搜索被告出生年龄
    def find_defenders_birthday(text,pattern):
        try:
            if re.search(pattern,text):
                piece = re.search(pattern,text).group().strip("，")
                text = re.sub("被告：.*，","",piece).strip("出生")
                return text
            else:
                return None
        except Exception as e:
            pass

    ## 立案时间
    def find_starttime(text,pattern):
        try:
            if re.search(pattern,text):
                piece = re.search(pattern,text).group().lstrip("，本院于").rstrip("立案后，")
                return piece
            else:
                return None
        except Exception as e:
            pass

    ## 事实和理由
    def find_facts(text,pattern):
        try:
            if re.search(pattern,text):
                piece = re.search(pattern,text).group()
                return piece
            else:
                return None
        except Exception as e:
            pass

    ## 被告辩称
    def find_defenders_words(text,pattern1,pattern2):
        try:
            if re.search(pattern1,text):
                piece = re.search(pattern1,text).group()
                return piece
            elif re.search(pattern2,text):
                piece = re.search(pattern2,text).group()
                return piece
            else:
                return None
        except Exception as e:
            pass

    ## 法院认定事实
    def find_court_facts(text,pattern1,pattern2,pattern3):
        try:
            if re.search(pattern1,text):
                piece = re.search(pattern1,text).group().rstrip("本院")
                return piece
            elif re.search(pattern2,text):
                piece = re.search(pattern2,text).group().rstrip("本院")
                return piece
            elif re.search(pattern3,text):
                piece = re.search(pattern3,text).group().rstrip("本院")
                return piece
            else:
                return None
        except Exception as e:
            pass

    ## 查找法条
    def find_laws(text,pattern3,pattern4):
        try:
            if re.search(pattern3,text):
                pieces_1 = re.findall(pattern3,text)
                short_pieces_1 = list(filter(lambda x:len(x) < 50,pieces_1))
                pieces_2 = re.findall(pattern4,text)
                short_pieces_2 = list(filter(lambda x:len(x) < 50,pieces_2))
                short_pieces_3 = short_pieces_1 + short_pieces_2
                return (short_pieces_1,short_pieces_2)
            else:
                return None
        except Exception as e:
            pass

    ## 大块文字专用
    def find_claims_or_blocks(text,pattern):
        try:
            if re.search(pattern,text):
                piece = re.search(pattern,text).group()
                return piece
            else:
                return None
        except Exception as e:
            pass
        
    ## 原告诉求
    def find_claims(text,pattern1,pattern2):
        try:
            if re.search(pattern1,text):
                piece = re.search(pattern1,text).group()
                return piece
            elif re.search(pattern2,text):
                piece = re.search(pattern2,text).group()
                return piece
        except Exception as e:
            pass
        
    ## 重复pattern，构造pattern list
    def repeat_pattern(pattern):
        pattern_lists = len(list(results)) * [pattern]
        return pattern_lists

    list_piece_t,list_corpus = [],[] 
    ### 是否这里可以改成(**kwargs)
    ### 原告
    
    plaintiffs = list(map(find_plaintiffs,corpus_none_n,repeat_pattern(pattern_plaintiff),
                          repeat_pattern(pattern_plaintiff_2),repeat_pattern(pattern_plaintiff_3)))

    ### 被告
    defenders = list(map(find_defender,corpus_none_n,repeat_pattern(pattern_defender),
                        repeat_pattern(pattern_defender_2)))
    
    ### 原告出生年龄
    plaintiffs_birthday = list(map(find_plaintiffs_birthday,corpus_none_n,repeat_pattern(pattern_plaintiff_birthday)))

    ###被告出生年龄
    defenders_birthday = list(map(find_defenders_birthday,corpus_none_n,repeat_pattern(pattern_defender_birthday)))

    ### 立案时间
    starttimes = list(map(find_starttime,corpus_none_n,repeat_pattern(pattern_starttime)))

    ### 诉讼请求
    claims = list(map(find_claims,corpus_none_n,repeat_pattern(pattern_claims),repeat_pattern(pattern_claims_2)))

    ### 事实与理由
    facts = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_facts)))

    ### 被告辩称
    defences = list(map(find_defender,corpus_none_n,repeat_pattern(pattern_defence),
                        repeat_pattern(pattern_defence_2)))
    
    ### 法院认定事实
    court_facts = list(map(find_court_facts,corpus_none_n,repeat_pattern(pattern_court_fact_1),
                          repeat_pattern(pattern_court_fact_2),repeat_pattern(pattern_court_fact_3)))

    ### 法院对事实的判定
    advises = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_court_advise)))

    ### 法院的态度
    attitudes = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_court_attitude)))

    ### 判决
    judgement = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_judgement)))

    ### 法条
    laws1 = list(map(find_laws,corpus_none_n,repeat_pattern(pattern_laws_3),
                        repeat_pattern(pattern_laws_4)))
    table_caipanwenshu = pd.DataFrame([plaintiffs,defenders,plaintiffs_birthday,defenders_birthday,
                                   starttimes,claims,facts,defences,court_facts,advises,attitudes,judgement,laws1]).T
    table_caipanwenshu.columns = ["原告","被告","原告出生时间","被告出生时间","立案时间","原告诉讼请求",
                              "原告事实与理由","被告辩称","法院认定事实","法院认为","法院意见","法院判决","法条1"]
    return table_caipanwenshu

##### 第四模块(法条统计模块:必须运行)

In [4]:
# 法条统计处理模块
laws_item_1,laws_item_2,list_temp_3 = [],[],[]
def generate_all_laws_1(laws):
    for item1,item2 in laws:
        for item in item1:
            laws_item_1.append(item)
    return laws_item_1

def generate_all_laws_2(laws):
    for item1,item2 in laws:
        for item in item2:
            laws_item_2.append(item)
    return laws_item_2

def split_all_laws(laws_item):
    for item in laws_item:     
        if "&lt;" in item:
            item = re.sub("&lt;","",item)
            item = re.sub("&gt;","",item)
        try:
            # 如果法条前面没有《..法》，就得识别并且用循环加
            list_temp,list_temp_2 = [],[]
            law_name = re.search("《.*?》",item).group()
            item_list = item.split("、")
            for item in item_list:
                if "《" not in item:
                    list_temp.append(law_name + item)
            item_list.extend(list_temp)

            # 提取前面有《..法》的法条
            for law in item_list:
                if "《" in law:
                    list_temp_2.append(law)
            list_temp_3.append(list_temp_2)
        except Exception as e:
            list_temp_3.append("[]")
    return list_temp_3

def count_laws_2(list_temp_3):
    law_corpus = []
    for law_list in list_temp_3:
        for item in law_list:
            law_corpus.append(item)
    return law_corpus

#### 抽取法条信息模块

In [ ]:
# 如果需要抽取判决中的法条
# ---------------------------------------------
def extract_laws(table):
    table_judgements = clean(table_clean["判决结果"]) # 这里就填判决结果
    table_judgements_extraction = table_judgements[["法院判决","法条1"]]
    laws = list(filter(lambda x:isinstance(x,tuple),table_judgements_extraction["法条1"]))
    laws_item = generate_all_laws_1(laws)
    list_temp_3 = split_all_laws(laws_item)
    law_corpus = count_laws_2(list_temp_3)
    table_final = pd.DataFrame([dict(Counter(law_corpus))]).T.sort_values(0,ascending = False)
    single_law = generate_all_laws_2(laws)
    table_single_laws = pd.DataFrame([dict(Counter(single_law))]).T.sort_values(0,ascending = False)
    return table_final,table_single_laws

if __name__ == "__main__":
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_clean = import_table(root)
    print("----------------------------------------------")
    print("******************表格已经导入******************")
    print("----------------------------------------------")

    table_final,table_single_laws = extract_laws(table_clean)
    table_final.columns = ["数量"]
    table_single_laws.columns = ["数量"]

    save_model_all_laws(table_final)
    save_model_single_laws(table_single_laws)
#.drop(["被告","原告出生时间","被告出生时间","立案时间","原告诉讼请求","原告事实与理由"],axis = 1)

#### 抽取其他信息模块

In [5]:
# 如果需要分开得到其他列中的信息
# ---------------------------------------------
def extract_infos(table):
    table_judgements = clean(table) # 这里可以填其他的
    return table_judgements

if __name__ == "__main__":
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_clean = import_table(root)
    print("----------------------------------------------")
    print("******************表格已经导入******************")
    print("----------------------------------------------")
    
    table_items = []
    for index in range(len(table_clean)):
        item = "".join(list(filter(lambda x:isinstance(x,str),table_clean.ix[index])))
        table_items.append(item)
    table_final = extract_infos(table_items)

----------------------------------------------
******************表格已经导入******************
----------------------------------------------


/Users/dfuser/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
